In [1]:
!pip install fasttext

     |████████████████████████████████| 68 kB 3.0 MB/s 
  Using cached pybind11-2.9.2-py2.py3-none-any.whl (213 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3146114 sha256=97f1069b57fc68f9eeae10dbf57cc88727fa2523f5c88fdac1c71bb5a190b274
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext


In [2]:
import numpy as np
from numpy import array
import pandas as pd
import gensim

import gzip
from urllib.request import urlopen

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Embedding,Activation, Dropout,Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from keras.layers import SimpleRNN
from keras.layers import Flatten

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns


In [3]:
#menyambungkan file ke google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_csv("/content/drive/My Drive/TA/pelabelan.csv")
df

,review_clean,rating,label
0,update sertifikat hilang kemarin update,5,Positif
1,vaksin kali alhamdulillah sehat terima kasih p...,5,Positif
2,atur tanggal nya bikin kesel,1,Negatif
3,pakai nomor nik vaksin tidak kalo tanggal vaks...,4,Positif
4,ngikut atas,5,Positif
...,...,...,...
49867,tolong kalo bikin sertifikat vaksin tanggal la...,5,Positif
49868,bingung aplikasi vaksin sertifikat beda vaksin...,2,Negatif
49869,aplikasi bagus sayang scan barcodenya radius s...,3,Netral
49870,vaksin masuk data ngga sertifikat nya,1,Negatif


In [5]:
# Inisialisasi variabel target
X = df['review_clean']
y = df['label']

In [6]:
# Membuka file zip FastText
# Ini pre-trained model
file = gzip.open(urlopen('https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.id.300.vec.gz'))
# Inisialisasi variabel untuk menyimpan vocab_and_vectors
vocab_and_vectors = {}
for line in file:
    values = line.split()
    word = values [0].decode('utf-8')
    vector = np.asarray(values[1:], dtype='float32')
    vocab_and_vectors[word] = vector

In [7]:
len(vocab_and_vectors)

2000000

In [8]:
tokenizer = Tokenizer(num_words=100, oov_token='*')

In [9]:
tokenizer.fit_on_texts(X.tolist())

In [10]:
seq = tokenizer.texts_to_sequences(X.tolist())

In [11]:
word_index = tokenizer.word_index
print(tokenizer.word_index)
print('Found %s unique tokens' % len(word_index))

{'*': 1, 'aplikasi': 2, 'vaksin': 3, 'ngga': 4, 'sertifikat': 5, 'nya': 6, 'masuk': 7, 'ya': 8, 'kali': 9, 'tolong': 10, 'tanggal': 11, 'muncul': 12, 'bantu': 13, 'data': 14, 'daftar': 15, 'baik': 16, 'lahir': 17, 'susah': 18, 'pakai': 19, 'buka': 20, 'update': 21, 'handphone': 22, 'mohon': 23, 'nomor': 24, 'bagus': 25, 'nik': 26, 'isi': 27, 'error': 28, 'banget': 29, 'kalo': 30, 'peduli': 31, 'cek': 32, 'email': 33, 'lindung': 34, 'login': 35, 'coba': 36, 'kasih': 37, 'bikin': 38, 'sih': 39, 'mudah': 40, 'kode': 41, 'orang': 42, 'pedulilindungi': 43, 'download': 44, 'nama': 45, 'ribet': 46, 'ulang': 47, 'scan': 48, 'salah': 49, 'verifikasi': 50, 'terimakasih': 51, 'akun': 52, 'guna': 53, 'terima': 54, 'tulis': 55, 'suruh': 56, 'periksa': 57, 'sulit': 58, 'baru': 59, 'sesuai': 60, 'gagal': 61, 'kirim': 62, 'hasil': 63, 'admin': 64, 'otp': 65, 'deh': 66, 'lokasi': 67, 'mana': 68, 'lengkap': 69, 'bintang': 70, 'lihat': 71, 'akses': 72, 'perintah': 73, 'kerja': 74, 'tuju': 75, 'klik': 76,

In [12]:
print(X[0])
print(seq[0])

update sertifikat hilang kemarin update
[21, 5, 77, 1, 21]


In [13]:
max_length = 500
for review_number in range(len(seq)):
  numberofwords=len(seq[review_number])
  if (numberofwords) > (max_length):
    max_length = numberofwords
print(max_length)

500


In [14]:
X = pad_sequences(seq, maxlen = max_length, padding='post')
print('Shape of data tensor:', X.shape)
print('Shape of label tensor: ', y.shape)

Shape of data tensor: (49872, 500)
Shape of label tensor:  (49872,)


In [15]:
# Mengubah y menjadi integer menggunakan LabelEncoder
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(y)
# Mengubah y yang sudah menjadi integer menggunakan OneHotEncoder
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
y = onehot_encoder.fit_transform(integer_encoded)

In [16]:
print('Shape of data tensor:', X.shape)
print('Shape of label tensor: ', y.shape)

Shape of data tensor: (49872, 500)
Shape of label tensor:  (49872, 3)


In [17]:
print(vocab_and_vectors.get('update'))
print(vocab_and_vectors.get('sertifikat').shape)
print(vocab_and_vectors.get('vaksin'))

[-0.0326  0.0522 -0.0375  0.0346  0.0823  0.0824  0.0424  0.021  -0.0443
 -0.0555  0.0093 -0.0802 -0.0614  0.0286  0.0341 -0.0425  0.0005  0.0591
  0.0048 -0.098  -0.0265  0.0114  0.0287  0.0618  0.0426 -0.0529  0.0113
  0.0298  0.0391  0.023   0.0636 -0.009   0.0239  0.0255 -0.0175  0.0469
 -0.0165  0.0358  0.0433  0.043   0.0585  0.0317 -0.0138 -0.0805  0.0055
  0.0063  0.0017 -0.1046  0.0246  0.02    0.0077 -0.0213  0.0098 -0.0497
  0.0718  0.0688  0.056  -0.0155 -0.0186  0.0117 -0.0358  0.0868 -0.0503
  0.0802  0.0755 -0.0113  0.005   0.086  -0.0127 -0.0069  0.0349  0.0232
 -0.0194 -0.0376  0.0277  0.042   0.0459 -0.0723  0.0351  0.0772  0.1018
 -0.0252  0.014  -0.073  -0.0309  0.0119  0.0241 -0.0509 -0.0911  0.0609
  0.0482  0.0112 -0.0416  0.0308 -0.0377 -0.0636  0.0363 -0.0322  0.0345
  0.0162 -0.0083  0.0268 -0.0007 -0.0206  0.0128 -0.0894  0.1814 -0.071
 -0.0102  0.0226 -0.0586 -0.075   0.006   0.1327 -0.0355  0.0273 -0.0144
 -0.0187  0.0609  0.0233 -0.1462 -0.0148 -0.0084 -0.

In [18]:
# Melakukan word embedding
# Membuat embedding matrix
embedding_matrix = np.random.random((len(word_index) + 1, 300))
unidentified_words = []
for word, i in word_index.items():
    embedding_vector = vocab_and_vectors.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
    else :
      unidentified_words.append(word)
print(" Completed!")

 Completed!


In [19]:
unidentified_words[0:10]

['pedulilindungi',
 'covid',
 'sinovac',
 'otpnya',
 'ehac',
 'prokes',
 'barkot',
 'sartifikat',
 'aplikasiny',
 'pedulindungi']

In [20]:
print("Embedding Matrix shape: ", embedding_matrix.shape)

Embedding Matrix shape:  (17615, 300)


In [21]:
embedding_matrix[0]

array([7.29287435e-01, 8.41109138e-01, 9.85294752e-02, 4.37462307e-01,
       6.24781233e-01, 1.35001109e-01, 4.42463388e-01, 4.34077784e-01,
       1.71936480e-01, 3.54356236e-01, 8.79560288e-01, 9.57768293e-01,
       4.39225716e-01, 9.90558713e-01, 7.16054508e-01, 4.09831213e-01,
       2.17865359e-01, 7.57382858e-01, 1.01975597e-01, 8.46398152e-01,
       4.60112065e-01, 8.49706639e-02, 4.11428897e-01, 7.22886727e-01,
       9.18427486e-01, 1.56098494e-01, 4.26751281e-01, 2.79099559e-01,
       4.59447460e-01, 1.65249897e-01, 5.52368609e-01, 7.49598771e-01,
       1.30243043e-02, 1.69854480e-01, 7.70070508e-01, 4.74665112e-01,
       5.89733024e-01, 1.62969092e-01, 5.66909735e-01, 1.66344692e-01,
       9.83207694e-03, 7.50479057e-01, 1.67099811e-01, 4.70774670e-03,
       9.70167173e-01, 9.52608785e-01, 3.49097574e-01, 8.37992355e-01,
       2.99669179e-02, 2.23478408e-01, 4.85817912e-01, 7.92708268e-01,
       6.72379391e-01, 5.92103977e-01, 5.18341103e-01, 6.40833092e-01,
      

In [22]:
# Split menjadi train dan test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, stratify=y, random_state=100)

In [23]:
# Menampilkan ukuran X_train, y_train, X_test, dan y_test
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(34910, 500) (34910, 3)
(14962, 500) (14962, 3)


In [24]:
print(X_train.ndim, y_train.ndim)
print(X_test.ndim, y_test.ndim)

2 2
2 2


In [25]:
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, Flatten
from tensorflow.keras.layers import Input, Dense, SimpleRNN
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD

In [39]:
# Create Model
model = Sequential()
# Embedding berisi parameter total kata yang ditokenisasi, panjang kalimat, dan dimensi embedding
# Embedding matrix dijadikan weight
model.add(Embedding(len(word_index)+1, 300, input_length = X.shape[1], weights=[embedding_matrix], trainable=False))
# Yang digunakan adalah convolutional neural network 1 dimensi
model.add(SimpleRNN(128, activation ='relu',return_sequences=True))
#model.add(SimpleRNN(256, activation ='relu'))
# Dropout supaya tidak overfitting
#model.add(Dense(5, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))
model.add(Flatten())
#model.add(GlobalAveragePooling1D())
# Dense adalah layer klasifikasi akhir
model.add(Dense(5, activation='sigmoid'))
model.build()
model.summary()
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

NameError: ignored

In [38]:
history= model.fit(X_train, y_train, epochs=5, batch_size=1024, validation_data=(X_test, y_test), verbose=2)

Epoch 1/5


InvalidArgumentError: ignored